In [1]:
%load_ext autoreload
%autoreload 2
import os

In [21]:
from robustnessgym.mosaic.contrib.wilds import WildsInputColumn, WildsDataPane
from robustnessgym.mosaic import DataPane
dp = WildsDataPane("fmow", root_dir="/home/common/datasets/", use_transform=True, split="val")

In [9]:
dp._data

{'input': WildsInputColumn(<wilds.datase...x7f4cc12c8790>),
 'y': NumpyArrayColumn(array([ 4,  4..., 34, 34, 34])),
 'meta': NumpyArrayColumn(array([[ 0, 1...[ 0,  9, 34]])),
 'index': ListColumn(['0', '1', '2', '3', '4', '5', ...])}

In [14]:
# Target column is a NumPy array: 
dp["y"]

NumpyArrayColumn(array([ 4,  4..., 34, 34, 34]))

In [16]:
dp["y"]

NumpyArrayColumn(array([ 5,  5..., 35, 35, 35]))

In [23]:
import torch
from robustnessgym.mosaic.contrib.wilds import WildsInputColumn, WildsDataPane
from spr.vision import Classifier
from robustnessgym.mosaic import DataPane
from spr.data.wilds import FMOW_CONFIG

classifier = Classifier(FMOW_CONFIG)
dp = WildsDataPane(
    "fmow", 
    root_dir="/home/common/datasets/", 
    use_transform=True,
    split="val"
)

classifier.eval()
classifier.to(0)

class ActivationExtractor:
    """Class for extracting activations of a targetted intermediate layer."""

    def __init__(self):
        self.activation = None

    def add_hook(self, module, input, output):
        self.activation = output
extractor = ActivationExtractor()
classifier.model.features[-1].register_forward_hook(extractor.add_hook)

@torch.no_grad()
def predict(batch: dict):
    out = torch.softmax(classifier(batch["input"].to(0)), axis=-1)
    return {
        "pred": out.cpu().numpy().argmax(axis=-1),
        "activation": extractor.activation.cpu().numpy()
    }

dp_w_preds = dp.update(
    function=predict,
    batch_size=128,
    batched=True,
    num_workers=8
)

loading weights from /home/common/datasets/fmow_v1.1/models/fmow_erm_ID_seed0/best_model.pth...
/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [24]:
dp_w_preds._data

{'input': WildsInputColumn(<wilds.datase...x7f4c7f739670>),
 'y': NumpyArrayColumn(array([ 4,  4..., 34, 34, 34])),
 'meta': NumpyArrayColumn(array([[ 0, 1...[ 0,  9, 34]])),
 'index': ListColumn(['0', '1', '2', '3', '4', '5', ...]),
 'pred': NumpyArrayColumn(array([ 4,  4..., 42, 61, 21])),
 'activation': NumpyArrayColumn(array([[[[0.0...dtype=float32))}

In [27]:
# Compute accuracy
(dp_w_preds["pred"] == dp_w_preds["y"]).mean()

NumpyArrayColumn(array([0.6196116]))

In [29]:
dp_w_preds["activation"].shape

(11483, 1024, 7, 7)